<a href="https://colab.research.google.com/github/rkudulis/Deep-Learning-Notebooks/blob/main/Fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>